In [1]:
import pandas as pd
import numpy as np

In [2]:
import psycopg2
conn=psycopg2.connect(database="ernie",user="wenxi",host="localhost",password="temp_ERNIE_1234")
conn.set_client_encoding('UTF8')
conn.autocommit=True
curs=conn.cursor()

### Theta_Plus Visualization

In [3]:
curs.execute("SET SEARCH_PATH TO theta_plus;")

In [54]:
node_size = pd.read_sql(
'''
WITH cte AS (SELECT a.cluster_no, a.scp, b.auid FROM theta_plus.imm1985_1995_cluster_scp_list_unshuffled a INNER JOIN public.scopus_authors b
ON a.scp = b.scp
WHERE cluster_no = 1673)
SELECT cte.auid, COUNT(*) as node_size FROM cte
GROUP BY cte.auid
ORDER BY cte.auid
''', conn)

In [55]:
node_size

,auid,node_size
0,6504247797,1
1,6505818722,2
2,6505825422,1
3,6505842916,1
4,6506013280,2
...,...,...
380,57202883565,1
381,57204499718,1
382,57210121267,1
383,57211000906,1


In [56]:
edge_size = pd.read_sql(
'''
WITH cte AS(
    SELECT a.scp, b.auid FROM theta_plus.imm1985_1995_cluster_scp_list_unshuffled a
    INNER JOIN public.scopus_authors b
               ON a.scp = b.scp
     WHERE a.cluster_no = 1673
     ORDER BY auid)
SELECT author_1, author_2, COUNT(*) FROM (
SELECT cte.scp, cte.auid AS author_1, c.auid AS author_2 FROM cte JOIN cte c ON cte.scp = c.scp AND cte.auid < c.auid 
ORDER BY author_1, author_2
) d
GROUP BY author_1, author_2
''', conn)

In [59]:
edge_size.sort_values('count')

,author_1,author_2,count
0,6504247797,6603909279,1
1045,7403870294,9249156000,1
1044,7403870232,56429533400,1
1043,7403870232,54400897900,1
1042,7403870232,24922661200,1
...,...,...,...
891,7102430622,9249156000,10
1068,8157312000,12239701300,10
916,7102430622,55752867400,10
887,7102430622,7103013737,12


In [66]:
import networkx as nx
from netwulf import visualize

G = nx.from_pandas_edgelist(edge_size, 'author_1', 'author_2')

df = node_size.set_index('auid')
df = df.loc[list(G.nodes)].reset_index()


i=0
# Set node 'size' attributes
for n, data in G.nodes(data=True):
    data['size'] = df['node_size'][i]
    i+=1
i=0
# Set link 'weight' attributes
for n1, n2, data in G.edges(data=True):
    data['weight'] = edge_size['count'][i]
    i+=1


visualize(G)

(None, None)

In [65]:
node_size.sort_values('node_size', ascending=False).head(10).reset_index().drop(columns='index')

,auid,node_size
0,7103013737,31
1,7102430622,17
2,7006139394,16
3,16935919000,11
4,8157312000,11
5,12239701300,11
6,9249156000,10
7,55752867400,10
8,6701663272,9
9,7103047807,8
